## Properties / Metadata
The document properties represent the fields that define the data elements you want to retrieve as well as the elements you choose to filter your results.  In addition, properties are at the heart of other Search parameters such as Navigation, Grouping, and Boosting.  There are hundreds of properties available for each View providing critical details needed to fulfill your searching requirements.

In order for you to be successful at search, you will need to understand how to access the available properties, how to determine what values they contain and what they mean.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.19.0 Refinitiv.Data.Content, 1.0.0-beta3

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Delivery.Request;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

24/01/2022 12:05:26 PM:DesktopSession. (State: DesktopSession is Pending)
24/01/2022 12:05:27 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
24/01/2022 12:05:27 PM:DesktopSession. (State: DesktopSession is Opened)


#### Metadata
MetaData is a convenient service that allows you to list all available properties within a View.  To properly build a metadata list for inspection, you can issue the following:

**Note**: The RDP Libraries for .Net does not currently provide a Content Layer interface for the Search MetaData service.  This will be coming soon.  As such, the code segment below will provide the interface at the Delivery layer.

In [5]:
// Specify the view
var response = MetaData.Definition(Search.View.CommodityQuotes).GetData();
response.Data.Table

AAACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
AACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
ABSMBSBondsRIC,String,False,False,False,False,False,False
ActivityDate,Date,True,True,True,False,False,False
ACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
AdmissionToTrading,Date,True,True,True,False,False,False
AliasRIC,String,True,False,False,True,True,False
AllATMOptionsContinuationChainsRIC,String,False,False,False,False,False,False
AllStrikesOptionsChainRIC,String,False,False,False,False,False,False
AlternateISIN,String,True,False,False,False,True,False
AnalyticRIC,String,True,False,False,False,True,False


#### Debugging (*_debugall*)
While MetaData will help list all properties available within a View, all properties will not be available for use.  That is, depending on the type of search you perform, will determine which properties will be populated - not all will be.  This is where users can use a technique to list all available properties that are populated, for the specific search requested.

In [6]:
// Note: It is critical you select one document top=1 because the amount of data return is very large
var response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                       .Top(1)   
                                                       .Select("_debugall")
                                                       .GetData();
Console.WriteLine(response.Data?.Raw);

{
  "Total": 46158,
  "Hits": [
    {
      "meta": {
        "_id": "37036",
        "_index": "eikonsearch_organisation_1.138.0",
        "_score": 2838808.0,
        "_timestamp": 1643037930430,
        "_timestamp_human": "2022-01-24T15:25:30.43Z",
        "_version": 1643036370000
      },
      "raw_source": {
        "RCSOrganisationTypeGenealogy": "OT:1",
        "SPsRatingOutlookX1XSPsRatingOutlookRank": "SPI/StableX1X0|SPI/StableX1X0",
        "RCSTRBCGenealogy": "B:161\\B:171\\B:172\\B:173",
        "IssuerID": "0x000019000050e3aa",
        "SicSchemeCode": "8742",
        "OfficersExist": true,
        "IssuerBondsExist": true,
        "CertificatesOfDepositCount": 0,
        "MktCapCompanyUsd": 116001125272,
        "TotalAssetsUsd": 155971000000,
        "GEMPermID": "4295904307",
        "SubjectBoost": "firm",
        "EEDBID": "14129",
        "SigDevExist": true,
        "SDCID": "3104001",
        "TotalAssets": 155971000000,
        "EstimatesExist": true,
        "

#### Understanding values using Navigators
Using the output from a debugging session, there are a number of common properties that are very useful when filtering your search.  For example, you may want to filter your results based on a specific category of data.  Using navigators will help us understand these categories.

Looking at our debug output above, one useful property: **SearchAllCategoryv3** will helps us understand all available categories of documents the query term 'IBM' was found.  To list these categories, use the following:

In [7]:
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(0)
                                                   .Navigators("SearchAllCategoryv3")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["SearchAllCategoryv3"]["Buckets"]);

[
  {
    "Label": "Other Quote",
    "Count": 27313
  },
  {
    "Label": "Warrants",
    "Count": 8406
  },
  {
    "Label": "Bond Pricing",
    "Count": 2024
  },
  {
    "Label": "Bonds",
    "Count": 1966
  },
  {
    "Label": "Options",
    "Count": 1503
  },
  {
    "Label": "Commercial Papers",
    "Count": 1308
  },
  {
    "Label": "Futures",
    "Count": 767
  },
  {
    "Label": "Mergers & Acquisitions",
    "Count": 733
  },
  {
    "Label": "Companies/Issuers",
    "Count": 680
  },
  {
    "Label": "Equities",
    "Count": 543
  },
  {
    "Label": "Credit Default Swaps",
    "Count": 281
  },
  {
    "Label": "Analyst",
    "Count": 241
  },
  {
    "Label": "Officers",
    "Count": 195
  },
  {
    "Label": "FX & Money",
    "Count": 65
  },
  {
    "Label": "Fixed Income Curves",
    "Count": 48
  },
  {
    "Label": "Other Instrument",
    "Count": 35
  },
  {
    "Label": "Mortgage Backed Securities",
    "Count": 12
  },
  {
    "Label": "Funds",
    "Count": 10
  

Using the above listing, we can narrow our result set based on these categories using a Filter.

#### Special Properties
The Search service includes a number of special properties that enhance the ability to find meaningful results. In addition to the special property *_debugall* used within the **select** parameter as outlined above, the following properties are available: 

In [8]:
// The 'code' specification will search across all codes, such as RICs, tickers, SEDOLs etc.  
// Primarily used within a filter but can be used as a selectable property.
response = Search.Definition(Search.View.SearchAll).Filter("code eq 'IBM'")
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",37036,,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",1097326,55839165994,IBM
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, MiFID Composite Cross Market Service",79345498,21475739068,IBMEUR.xbo
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, MiFID Composite Cross Market Service",131770829,21523029548,IBMUSD.xbo
QUOTExEQUITY,"International Business Machines Corp, Depository Receipt, Bolsa de Comercio de Buenos Aires",1006058,55838283585,IBM.BA
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, New York Stock Exchange",1090370,55838323096,IBM.N
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NASDAQ INTERMARKET",765744,55835333953,IBM.TH
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, Cboe BZX Exchange",53703996,55845629228,IBM.Z
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, Cboe EDGX Exchange",73533774,21475059739,IBM.DG
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Arca",753753,55837129318,IBM.P


In [9]:
// The 'name' specification will search across all names, such as common name, legal name, issuer name, etc. 
// Primarily used within a filter but can be used as a selectable property.
response = Search.Definition(Search.View.SearchAll).Filter("name eq 'IBM'")
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",37036,,
ORGANISATION,"Banco IBM SA, Private Company",76208,,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",1097326,55839165994,IBM
ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",18062670,,
QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",48924732,21481052421,0#IBMF:
QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",259118763,21612423771,0#IBMDF:
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",49450681,21481052892,IBMFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",50092347,21481053949,IBMFc2
QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",260213021,21613372305,IBMDFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",50092348,21481053950,IBMFc3


In [10]:
// The special character underscore (_) is a convenient token that can be used to select the default Properties.  
// This will allow users to quickly select the default values plus any other properties when testing out your search.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(1000)
                                                   .Select("_, SearchAllCategoryv3")
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",37036,Companies/Issuers,,
ORGANISATION,"Banco IBM SA, Private Company",76208,Companies/Issuers,,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",1097326,Equities,55839165994,IBM
ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",18062670,Companies/Issuers,,
QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",48924732,Futures,21481052421,0#IBMF:
QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",259118763,Futures,21612423771,0#IBMDF:
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",49450681,Futures,21481052892,IBMFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",50092347,Futures,21481053949,IBMFc2
QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",260213021,Futures,21613372305,IBMDFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",50092348,Futures,21481053950,IBMFc3
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex",50092349,Futures,21481053951,IBMFc4


#### Common Properties
As you become more familiar with search, you will soon recognize there will be a number of very useful properties that will act as your "go-tos" when filtering or selecting fields.

In [11]:
// DocumentTitle
// As you've seen in some of the above output, this property is on by default when you want to display your search results.  
// The DocumentTitle property is a quick way to describe the content retrieved for each matching document.  
// It is made up of the following child properties: 
//    'DTSubjectName', 'DTSimpleType', 'DTSource', and 'DTCharacteristics'.
var select = "DocumentTitle, DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.SearchAll).Select(select)
                                                   .Query("IBM")
                                                   .Top(100)
                                                   .GetData();
response.Data.Table

"International Business Machines Corp, Public Company",International Business Machines Corp,Public Company,,
"Banco IBM SA, Private Company",Banco IBM SA,Private Company,,
"International Business Machines Corp, Ordinary Share, NYSE Consolidated",International Business Machines Corp,Ordinary Share,NYSE Consolidated,
"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4,Private Company,,
"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Chain Contract,Equity Future,Eurex,USD
"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",Euronext Amsterdam IBM Dividend Future Chain Contracts,Equity Future,Euronext Amsterdam,USD
"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 1,Equity Future,Eurex,USD
"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 2,Equity Future,Eurex,USD
"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1,Equity Future,Euronext Amsterdam,USD
"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 3,Equity Future,Eurex,USD
"Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 4,Equity Future,Eurex,USD


In [12]:
// SearchAllCategoryv3
// When trying to narrow down your search to specific categories, this property can be extremely useful.  
// When you apply a Navigator on this property, it will return a list of ~40 categories, depending on the view. 
// Categories include asset-related instruments such as:
//     'Equities', 'Futures' , 'Options', 'Bonds', 'Commodities' 
//     but can also include things like: 'People', 'Analyst', 'Officers'.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Filter("SearchAllCategoryv3 eq 'People'")
                                                   .GetData();
response.Data.Table

PERSON,John Denham - IsUnknown,34414661717,34414661717
PERSON,Charlie Maxfield - IsUnknown,34414669683,34414669683
PERSON,Claudia Barclay - IsUnknown,34414721056,34414721056
PERSON,Robert Ploder - IsUnknown,34414738667,34414738667
PERSON,Tom Cadigan - IsUnknown,34414743082,34414743082
PERSON,R. L. Jay Vivian - IsUnknown,34414808510,34414808510
PERSON,William E. Baker - IsUnknown,34415092341,34415092341


In [13]:
// CodeResolutionBase
// Represents the most significant codes defined within the document.  For example, when searching for a symbol such 
// as ISIN, SEDOL, RIC, etc, filtering against this property will significantly reduce the matches and only bring 
// back the most relevant results.
response = Search.Definition(Search.View.SearchAll).Filter("CodeResolutionBase eq 'US046353AQ14'")
                                                   .Select("DocumentTitle, ISIN, SEDOL, RIC")
                                                   .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,046353AQ1=


In [14]:
// Conversely, if we were to choose the more generic term 'code', would result in a wider range of matches.
response = Search.Definition(Search.View.SearchAll).Filter("code eq 'US046353AQ14'")
                                                   .Select("DocumentTitle, ISIN, SEDOL, RIC")
                                                   .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,046353AQ1=
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS",,BF4SNM7,046353AQ1=1M
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS L2",,BF4SNM7,046353AQ1=2M
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, BARCLAYS TRADEWEB PRICE",,BF4SNM7,US046353AQ14=BXTW
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, CREDIT SUISSE TRADEWEB PRICE",,BF4SNM7,US046353AQ14=CSTW
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS COMPOSITE PLUS",,BF4SNM7,US046353AQ14=MKCP
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB APA",,BF4SNM7,US046353AQ14=TBEA
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TDS TRADEWEB PRICE",,BF4SNM7,US046353AQ14=TDTW
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",,BF4SNM7,US046353AQ14=TE
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",,BF4SNM7,046353AQ1=TWEB


#### Common Properties - RCS (Reuters Classification Scheme)
Throughout many documents, you will notice a number of RCS fields that are popular when selecting and filtering.

*Note* - I'll use navigators to demonstrate the values of these common properties.

In [15]:
// RCSAssetCategoryLeaf
// Specific to assets, this property will categorize the results into a readable identifier such as: 'Bond', 
// 'Commodity Future', 'Commercial Paper', etc.  Where the property SearchAllCategoryv3 provides general categories, 
// this property will be more specific to assets.  Pull out the top 10.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSAssetCategoryLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"]);

[
  {
    "Label": "Barrier Warrant",
    "Count": 19997048
  },
  {
    "Label": "Bond",
    "Count": 17041426
  },
  {
    "Label": "Commodity Future",
    "Count": 4898200
  },
  {
    "Label": "Traditional Warrant",
    "Count": 4275517
  },
  {
    "Label": "Equity Cash Option",
    "Count": 3962020
  },
  {
    "Label": "Interest Rate Future Spread",
    "Count": 3089242
  },
  {
    "Label": "Commodity Spread",
    "Count": 2922667
  },
  {
    "Label": "Equity Future",
    "Count": 2808603
  },
  {
    "Label": "Mortgage-Backed Security",
    "Count": 2799771
  },
  {
    "Label": "Certificate of Deposit",
    "Count": 2329360
  },
  {
    "Label": "Commercial Paper",
    "Count": 2157706
  },
  {
    "Label": "Warrant",
    "Count": 2038002
  },
  {
    "Label": "Commodity Future Option",
    "Count": 2000471
  },
  {
    "Label": "Commodity Spot",
    "Count": 1964946
  },
  {
    "Label": "Ordinary Share",
    "Count": 1758238
  },
  {
    "Label": "Commodity Forward",
    "

In [16]:
// RCSCountryLeaf
// RCSIssuerCountryLeaf
// RCSExchangeCountryLeaf
// Specific to some assets, will allow you to narrow your search to focus on a specific country.  Pull out the top 20.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSCountryLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSCountryLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 12760982
  },
  {
    "Label": "Eurobond",
    "Count": 4190325
  },
  {
    "Label": "China (Mainland)",
    "Count": 1250742
  },
  {
    "Label": "Colombia",
    "Count": 1017866
  },
  {
    "Label": "South Korea",
    "Count": 563940
  },
  {
    "Label": "Germany",
    "Count": 483626
  },
  {
    "Label": "Canada",
    "Count": 406768
  },
  {
    "Label": "France",
    "Count": 246619
  },
  {
    "Label": "India",
    "Count": 198070
  },
  {
    "Label": "Japan",
    "Count": 163722
  },
  {
    "Label": "Italy",
    "Count": 134978
  },
  {
    "Label": "Chile",
    "Count": 131886
  },
  {
    "Label": "Australia",
    "Count": 103792
  },
  {
    "Label": "Switzerland",
    "Count": 65643
  },
  {
    "Label": "Brazil",
    "Count": 65338
  },
  {
    "Label": "Russia",
    "Count": 61988
  },
  {
    "Label": "Spain",
    "Count": 61924
  },
  {
    "Label": "Austria",
    "Count": 58192
  },
  {
    "Label": "Norway",
    

In [17]:
// RCSCurrencyLeaf
// Specific to some assets, will allow you to narrow your search to focus on a specific currency.  Just pull out the top 20.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSCurrencyLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSCurrencyLeaf"]["Buckets"]);

[
  {
    "Label": "Euro",
    "Count": 33152880
  },
  {
    "Label": "US Dollar",
    "Count": 33012017
  },
  {
    "Label": "Chinese Yuan",
    "Count": 2323606
  },
  {
    "Label": "US Cent",
    "Count": 1539850
  },
  {
    "Label": "Colombian Peso",
    "Count": 1256754
  },
  {
    "Label": "Japanese Yen",
    "Count": 1122978
  },
  {
    "Label": "Hong Kong Dollar",
    "Count": 1063353
  },
  {
    "Label": "Canadian Dollar",
    "Count": 941228
  },
  {
    "Label": "Swedish Krona",
    "Count": 911093
  },
  {
    "Label": "British Pound",
    "Count": 827747
  },
  {
    "Label": "Swiss Franc",
    "Count": 822660
  },
  {
    "Label": "Indian Rupee",
    "Count": 819101
  },
  {
    "Label": "South Korean Won",
    "Count": 749571
  },
  {
    "Label": "Australian Dollar",
    "Count": 494068
  },
  {
    "Label": "British Penny",
    "Count": 457090
  },
  {
    "Label": "Hungarian Forint",
    "Count": 385385
  },
  {
    "Label": "Norwegian Krone",
    "Count": 3775

In [18]:
// RCSTRBC2012Leaf
// Represents the industry sector.  For example, if I query for Apple Computers, I will get many different 
// industry sectors such as: 'Computer Hardware (NEC)', 'Laptop & Desktop Computers', 'Phones & Smart Phones', etc.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSTRBC2012Leaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSTRBC2012Leaf"]["Buckets"]);

[
  {
    "Label": "Banks (NEC)",
    "Count": 4215344
  },
  {
    "Label": "Corporate Financial Services (NEC)",
    "Count": 3059738
  },
  {
    "Label": "Corporate Banks",
    "Count": 1435794
  },
  {
    "Label": "Retail & Mortgage Banks",
    "Count": 1290885
  },
  {
    "Label": "Investment Banking & Brokerage Services (NEC)",
    "Count": 722590
  },
  {
    "Label": "Public Finance Activities",
    "Count": 657860
  },
  {
    "Label": "Government & Government Finance (NEC)",
    "Count": 412284
  },
  {
    "Label": "Consumer Lending (NEC)",
    "Count": 407075
  },
  {
    "Label": "Investment Holding Companies (NEC)",
    "Count": 375523
  },
  {
    "Label": "Investment Management & Fund Operators (NEC)",
    "Count": 346476
  },
  {
    "Label": "Construction & Engineering (NEC)",
    "Count": 326361
  },
  {
    "Label": "Electric Utilities (NEC)",
    "Count": 300743
  },
  {
    "Label": "Wealth Management",
    "Count": 295382
  },
  {
    "Label": "Diversified Inv

In [19]:
// RCSTRBC2012Leaf
// RCSTRBC2012Genealogy
// RCSTRBC2012Name
// Provide a breakdown of the hierarchy defined for many of these RCS attributes.  You can see the mapping between the 
// hierarchy defined with the Genealogy property and Name property.  
// For example, the code 'B:262' maps to the name 'Banking & Investment Services'.
response = Search.Definition(Search.View.SearchAll).Filter("RCSTRBC2012Leaf eq 'Banks'")
                                                   .Top(0)
                                                   .Navigators("RCSTRBC2012Leaf, RCSTRBC2012Genealogy, RCSTRBC2012Name")
                                                   .GetData();
Console.WriteLine(response.Data.Raw);

{
  "Total": 6995429,
  "Hits": [],
  "Navigators": {
    "RCSTRBC2012Genealogy": {
      "Buckets": [
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1615",
          "Count": 4215344
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1616",
          "Count": 1435794
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1617",
          "Count": 1290947
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1619",
          "Count": 28318
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:131\\B:1629",
          "Count": 20216
        },
        {
          "Label": "B:261\\B:262\\B:263\\B:207\\B:1638",
          "Count": 3946
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1620",
          "Count": 576
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1618",
          "Count": 288
        }
      ]
    },
    "RCSTRBC2012Leaf": {
      "Buckets": [
        {
    

The above output includes 3 separate sections: RCSTRBC2012Genealogy, RCSTRBC2012Leaf and RCSTRBC2012Name.  For each you can see a breakdown ot the buckets.